In [1]:
library(mgcv)

Loading required package: nlme

This is mgcv 1.8-35. For overview type 'help("mgcv-package")'.



In [2]:
# gamsel
library(gamsel)
df <- read.csv('data/joined_v5.csv')
predictors = c('clogp_corona',
              'mon_corona_mw',
              'mon_corona_apol',
              'mon_corona_mv',
              'mon_corona_psa',
              'dp_corona',
              'corona_mw_total',
              'corona_mv_total',
              'clogp_core',
              'mon_core_mw',
              'mon_core_apol',
              'mon_core_mv',
              'mon_core_psa',
              'dp_core',
              'core_mw_total',
              'core_mv_total',
              'ratio_mass',
              'ratio_vol',
              'conc',
              'ph',
              'salt',
              'charged',
              'temp')

targets <- c("sphere","worm", "vesicle", "other")
# temp = df[ , names(df) %in% predictors ]
# # temp_name = append(predictors, "sphere")
# # temp = df[ , names(df) %in% predictors]
# y <- df[c("sphere","worm", "vesicle", "other")]
# gamsel.out=gamsel(temp, c(y$other),degrees=rep(1), family="binomial")
# a = predict(gamsel.out, temp, index=1, type="response") <=0.5

# sum((a == c(y$sphere))^2)/length(c(y$sphere))

Warning message:
“package ‘gamsel’ was built under R version 4.0.5”
Loaded gamsel 1.8-2




In [3]:
modify_data <- function(data){
    for (col in colnames(data)){
        if (nrow(unique(data[col])) == 1){
            data[col] <- NULL
        }
    }
    return (data)
}

neglikehood<-function(p, y){
    result <- (-log(p)*y - (1-y)*log(1-p))
    return (result)
}

mophology_loglikelihood<-function(mophology, data){
    result = c()
    for (i in 1:length(mophology)){
        neg = neglikehood(mophology[i], data[i])
        result <- append(result, neg)
    }
    return (result)
}

gam_phase <- function(predictors, targets, df, train_indx, test_indx){
    x_train <- df[train_indx, names(df) %in% predictors]
    y_train <- df[train_indx, names(df) %in% targets]
    
    x_test <- df[test_indx, names(df) %in% predictors]
    y_test <- df[test_indx, names(df) %in% targets]
    
#     x_train <- modify_data(x_train)
#     x_test <- x_test[, names(x_test) %in% colnames(x_train)]
    
    ######## model part
    # sphere
#     gamsel.sphere=gamsel(x_train, c(y_train$sphere),degrees=rep(1), family="binomial")
    gamsel.sphere=gam(sphere~clogp_corona + mon_corona_mw + mon_corona_apol + mon_corona_mv +
                        mon_corona_psa + dp_corona + corona_mw_total + corona_mv_total + clogp_core +
                        mon_core_mw + mon_core_apol + mon_core_mv + mon_core_psa + dp_core + core_mw_total + 
                        core_mv_total + ratio_mass + ratio_vol + conc + ph + salt + charged + temp, 
                        data=df[train_indx,], family=binomial("logit"))
    test_sphere_prob = predict(gamsel.sphere, x_test, index=1, type="response")
    test_sphere = test_sphere_prob >=0.5
    
    train_sphere_prob = predict(gamsel.sphere, x_train, index=1, type="response")
    train_sphere = train_sphere_prob >=0.5
    
    # worm
#     gamsel.worm=gamsel(x_train, c(y_train$worm),degrees=rep(1), family="binomial")
    gamsel.worm=gam(worm~clogp_corona + mon_corona_mw + mon_corona_apol + mon_corona_mv +
                        mon_corona_psa + dp_corona + corona_mw_total + corona_mv_total + clogp_core +
                        mon_core_mw + mon_core_apol + mon_core_mv + mon_core_psa + dp_core + core_mw_total + 
                        core_mv_total + ratio_mass + ratio_vol + conc + ph + salt + charged + temp, 
                        data=df[train_indx,], family=binomial("logit"))
    test_worm_prob = predict(gamsel.worm, x_test, index=1, type="response")
    test_worm = test_worm_prob >=0.5
    
    train_worm_prob = predict(gamsel.worm, x_train, index=1, type="response")
    train_worm = train_worm_prob >=0.5
    
    
    # vesicle
#     gamsel.vesicle=gamsel(x_train, c(y_train$vesicle),degrees=rep(1), family="binomial")
    gamsel.vesicle=gam(vesicle~clogp_corona + mon_corona_mw + mon_corona_apol + mon_corona_mv +
                        mon_corona_psa + dp_corona + corona_mw_total + corona_mv_total + clogp_core +
                        mon_core_mw + mon_core_apol + mon_core_mv + mon_core_psa + dp_core + core_mw_total + 
                        core_mv_total + ratio_mass + ratio_vol + conc + ph + salt + charged + temp, 
                        data=df[train_indx,], family=binomial("logit"))
    test_vesicle_prob = predict(gamsel.vesicle, x_test, index=1, type="response")
    test_vesicle = test_vesicle_prob >=0.5
    
    train_vesicle_prob = predict(gamsel.vesicle, x_train, index=1, type="response")
    train_vesicle = train_vesicle_prob >=0.5
    
    # other
#     gamsel.other=gamsel(x_train, c(y_train$other),degrees=rep(1), family="binomial")
    gamsel.other=gam(other~clogp_corona + mon_corona_mw + mon_corona_apol + mon_corona_mv +
                        mon_corona_psa + dp_corona + corona_mw_total + corona_mv_total + clogp_core +
                        mon_core_mw + mon_core_apol + mon_core_mv + mon_core_psa + dp_core + core_mw_total + 
                        core_mv_total + ratio_mass + ratio_vol + conc + ph + salt + charged + temp, 
                        data=df[train_indx,], family=binomial("logit"))
    test_other_prob = predict(gamsel.other, x_test, index=1, type="response")
    test_other = test_other_prob >=0.5
    
    train_other_prob = predict(gamsel.other, x_train, index=1, type="response")
    train_other = train_other_prob >=0.5
    
    #################################### accuracy part ####################################
    ######### Joint
    # test
    test_joint_lst <- c()
    
    for (i in 1:nrow(x_test)){
        test_joint = test_sphere[i] == y_test$sphere[i] & test_vesicle[i] == y_test$vesicle[i] &
            test_worm[i] == y_test$worm[i] & test_other[i] == y_test$other[i]
        test_joint_lst = append(test_joint_lst, as.numeric(isTRUE(test_joint)))
    }
    # train
    train_joint_lst <- c()
    
    for (i in 1:nrow(x_train)){
        train_joint = train_sphere[i] == y_train$sphere[i] & train_vesicle[i] == y_train$vesicle[i] &
            train_worm[i] == y_train$worm[i] & train_other[i] == y_train$other[i]
        train_joint_lst = append(train_joint_lst, as.numeric(isTRUE(train_joint)))
    }
    
    ######## Individual
    # test
    test_sphere_inds <- c()
    test_vesicle_inds <- c()
    test_worm_inds <- c()
    test_other_inds <- c()
    
    for (i in 1:nrow(x_test)){
        test_sphere_ind = test_sphere[i] == y_test$sphere[i]
        test_vesicle_ind = test_vesicle[i] == y_test$vesicle[i]
        test_worm_ind = test_worm[i] == y_test$worm[i]
        test_other_ind = test_other[i] == y_test$other[i]
        
        test_sphere_inds = append(test_sphere_inds, as.numeric(isTRUE(test_sphere_ind)))
        test_vesicle_inds = append(test_vesicle_inds, as.numeric(isTRUE(test_vesicle_ind)))
        test_worm_inds = append(test_worm_inds, as.numeric(isTRUE(test_worm_ind)))
        test_other_inds = append(test_other_inds, as.numeric(isTRUE(test_other_ind)))
    }
    
    # train
    train_sphere_inds <- c()
    train_vesicle_inds <- c()
    train_worm_inds <- c()
    train_other_inds <- c()
    
    for (i in 1:nrow(x_train)){
        train_sphere_ind = train_sphere[i] == y_train$sphere[i]
        train_vesicle_ind = train_vesicle[i] == y_train$vesicle[i]
        train_worm_ind = train_worm[i] == y_train$worm[i]
        train_other_ind = train_other[i] == y_train$other[i]
        
        train_sphere_inds = append(train_sphere_inds, as.numeric(isTRUE(train_sphere_ind)))
        train_vesicle_inds = append(train_vesicle_inds, as.numeric(isTRUE(train_vesicle_ind)))
        train_worm_inds = append(train_worm_inds, as.numeric(isTRUE(train_worm_ind)))
        train_other_inds = append(train_other_inds, as.numeric(isTRUE(train_other_ind)))
    }
    
    #################################### loglikelihood part ####################################
    ######## Individual
    # test
    test_sphere_log <- c()
    test_worm_log <- c()
    test_vesicle_log <- c()
    test_other_log <- c()
    
    test_sphere_log <- mophology_loglikelihood(test_sphere_prob, y_test$sphere)
    test_worm_log <- mophology_loglikelihood(test_worm_prob, y_test$worm)
    test_vesicle_log <- mophology_loglikelihood(test_vesicle_prob, y_test$vesicle)
    test_other_log <- mophology_loglikelihood(test_other_prob, y_test$other)

    # train
    train_sphere_log <- mophology_loglikelihood(train_sphere_prob, y_train$sphere)
    train_worm_log <- mophology_loglikelihood(train_worm_prob, y_train$worm)
    train_vesicle_log <- mophology_loglikelihood(train_vesicle_prob, y_train$vesicle)
    train_other_log <- mophology_loglikelihood(train_other_prob, y_train$other)

    
    ######## Joint
    # test
    test_joint_log = c(test_sphere_log) + c(test_worm_log) + c(test_vesicle_log) + c(test_other_log)
    
    # train
    train_joint_log = c(train_sphere_log) + c(train_worm_log) + c(train_vesicle_log) + c(train_other_log)
    
    df_joint_train = data.frame(
        train_joint_acc = train_joint_lst,        
        train_joint_log = train_joint_log
        )
    
    df_joint_test = data.frame(
        test_joint_acc = test_joint_lst,
        test_joint_log = test_joint_log
        )
    
    
    df_train_inds = data.frame( 
        train_sphere_acc = train_sphere_inds,
        train_vesicle_acc = train_vesicle_inds,
        train_worm_acc = train_worm_inds,
        train_other_acc = train_other_inds,
        
        train_sphere_prob = train_sphere_prob,
        train_worm_prob = train_worm_prob,
        train_vesicle_prob = train_vesicle_prob,
        train_other_prob = train_other_prob,
        
        train_sphere_log = train_sphere_log,
        train_worm_log = train_worm_log,
        train_vesicle_log = train_vesicle_log,
        train_other_log = train_other_log
        )
    
    df_test_inds = data.frame( 
        test_sphere_acc = test_sphere_inds,
        test_vesicle_acc = test_vesicle_inds,
        test_worm_acc = test_worm_inds,
        test_other_acc = test_other_inds,
        
        test_sphere_prob = test_sphere_prob,
        test_worm_prob = test_worm_prob,
        test_vesicle_prob = test_vesicle_prob,
        test_other_prob = test_other_prob,
        
        test_sphere_log = test_sphere_log,
        test_worm_log = test_worm_log,
        test_vesicle_log = test_vesicle_log,
        test_other_log = test_other_log
        )
    
#     results <- list('train' = train_lst, 'test' = test_lst,)
    results <-list('joint_train'=df_joint_train, 'ind_train' = df_train_inds,
                  'joint_test'=df_joint_test, 'ind_test' = df_test_inds)
    return (results)
}

```
gam_indi_phase <- function(predictors, target, df, train_indx, test_indx){
    train_lst = c()
    test_lst = c()
    x_train <- df[train_indx, names(df) %in% predictors]
    y_train <- df[train_indx, names(df) %in% targets]
    
    x_test <- df[test_indx, names(df) %in% predictors]
    y_test <- df[test_indx, names(df) %in% targets]
    
    x_train <- modify_data(x_train)
    x_test <- x_test[, names(x_test) %in% colnames(x_train)]

    gamsel.model=gamsel(x_train, c(y_train),degrees=rep(1), family="binomial")
    test_prob = predict(gamsel.model, x_test, index=1, type="response")
    test_label = test_prob >=0.5
    
    train_prob = predict(gamsel.model, x_train, index=1, type="response")
    train_label = train_prob >=0.5
    
    for (i in 1:nrow(x_test)){
        test_code = test_label[i] == y_test[i]
        test_lst = append(test_lst, as.numeric(isTRUE(test_code)))
        
    }
    
    for (i in 1:nrow(x_train)){
        train_code = train_label[i] == y_train[i]
        train_lst = append(train_lst, as.numeric(isTRUE(train_code)))
    }
    
    results <- list('train_joint_acc' = train_joint_lst, 'test_joint_acc' = test_joint_lst,
                   )
    return (results)
}
```

In [9]:
data_index <- read.csv('Extrapolation_ind_index.csv')
joint_train_acc_lst <- c()
joint_test_acc_lst <- c()
joint_train_logloss_lst <- c()
joint_test_logloss_lst <- c()

joint_train_acc_var <- c()
joint_test_acc_var <- c()
joint_train_logloss_var <- c()
joint_test_logloss_var <- c()

ind_train_sphere_acc_lst <- c()
ind_train_sphere_logloss_lst <- c()
ind_train_sphere_acc_var  <- c()
ind_train_sphere_logloss_var  <- c()

ind_train_worm_acc_lst <- c()
ind_train_worm_logloss_lst <- c()
ind_train_worm_acc_var  <- c()
ind_train_worm_logloss_var  <- c()

ind_train_vesicle_acc_lst <- c()
ind_train_vesicle_logloss_lst <- c()
ind_train_vesicle_acc_var <- c() 
ind_train_vesicle_logloss_var <- c() 

ind_train_other_acc_lst <- c()
ind_train_other_logloss_lst <- c()
ind_train_other_acc_var <- c() 
ind_train_other_logloss_var <- c() 

ind_test_sphere_acc_lst <- c()
ind_test_sphere_logloss_lst <- c()
ind_test_sphere_acc_var  <- c()
ind_test_sphere_logloss_var  <- c()

ind_test_worm_acc_lst <- c()
ind_test_worm_logloss_lst <- c()
ind_test_worm_acc_var  <- c()
ind_test_worm_logloss_var  <- c()

ind_test_vesicle_acc_lst <- c()
ind_test_vesicle_logloss_lst <- c()
ind_test_vesicle_acc_var <- c() 
ind_test_vesicle_logloss_var <- c() 

ind_test_other_acc_lst <- c()
ind_test_other_logloss_lst <- c()
ind_test_other_acc_var <- c() 
ind_test_other_logloss_var <- c() 

# data_index[1, 'train']
for (i in 1:nrow(data_index)){
    train_indx = c(as.numeric(c(strsplit(data_index[i, 'train'], ", "))[[1]])) # train index
    test_indx =  c(as.numeric(c(strsplit(data_index[i, 'test'], ", "))[[1]]))
    total_length = length(train_indx) + length(test_indx)
    
    result <- gam_phase(predictors=predictors, targets=targets, df=df, train_indx=train_indx, test_indx=test_indx)
    
    ########joint
    joint_train_acc_lst <- c(joint_train_acc_lst, mean(result$joint_train$train_joint_acc))
    joint_test_acc_lst <- c(joint_test_acc_lst, mean(result$joint_test$test_joint_acc))
    
    joint_train_logloss_lst <- c(joint_train_logloss_lst, mean(result$joint_train$train_joint_log))
    joint_test_logloss_lst <- c(joint_test_logloss_lst, mean(result$joint_test$test_joint_log))
    
    ##### var lst
    joint_train_acc_var <- c(joint_train_acc_var, var(result$joint_train$train_joint_acc) * length(train_indx)/total_length)
    if (length(result$joint_test$test_joint_acc) != 1){
        joint_test_acc_var <- c(joint_test_acc_var, var(result$joint_test$test_joint_acc) * length(test_indx)/total_length)
        joint_test_logloss_var <- c(joint_test_logloss_var, var(result$joint_test$test_joint_log) * length(test_indx)/total_length)

    }
    
    joint_train_logloss_var <- c(joint_train_logloss_var, var(result$joint_train$train_joint_log)* length(train_indx)/total_length)
    
    
    # EXTRAPOLATION IS DIFFERENT IN LOGLOSS
    ########individual
    ind_train_sphere_acc_lst <- c(ind_train_sphere_acc_lst, mean(result$ind_train$train_sphere_acc))
    ind_train_sphere_logloss_lst <- c(ind_train_sphere_logloss_lst, mean(result$ind_train$train_sphere_log))
    ind_train_sphere_acc_var <- c(ind_train_sphere_acc_var, var(result$ind_train$train_sphere_acc) * length(train_indx)/total_length)
    ind_train_sphere_logloss_var <- c(ind_train_sphere_logloss_var, var(result$ind_train$train_sphere_log)* length(train_indx)/total_length)
    
    ind_train_vesicle_acc_lst <- c(ind_train_vesicle_acc_lst, mean(result$ind_train$train_vesicle_acc))
    ind_train_vesicle_logloss_lst <- c(ind_train_vesicle_logloss_lst, mean(result$ind_train$train_vesicle_log))
    ind_train_vesicle_acc_var <- c(ind_train_vesicle_acc_var, var(result$ind_train$train_vesicle_acc)* length(train_indx)/total_length)
    ind_train_vesicle_logloss_var <- c(ind_train_vesicle_logloss_var, var(result$ind_train$train_vesicle_log)* length(train_indx)/total_length)
    
    ind_train_worm_acc_lst <- c(ind_train_worm_acc_lst, mean(result$ind_train$train_worm_acc))
    ind_train_worm_logloss_lst <- c(ind_train_worm_logloss_lst, mean(result$ind_train$train_worm_log))
    ind_train_worm_acc_var <- c(ind_train_worm_acc_var, var(result$ind_train$train_worm_acc)* length(train_indx)/total_length)
    ind_train_worm_logloss_var <- c(ind_train_worm_logloss_var, var(result$ind_train$train_worm_log)* length(train_indx)/total_length)
    
    ind_train_other_acc_lst <- c(ind_train_other_acc_lst, mean(result$ind_train$train_other_acc))
    ind_train_other_logloss_lst <- c(ind_train_other_logloss_lst, mean(result$ind_train$train_other_log))
    ind_train_other_acc_var <- c(ind_train_other_acc_var, var(result$ind_train$train_other_acc)* length(train_indx)/total_length)
    ind_train_other_logloss_var <- c(ind_train_other_logloss_var, var(result$ind_train$train_other_log)* length(train_indx)/total_length)
    
    ind_test_sphere_acc_lst <- c(ind_test_sphere_acc_lst, mean(result$ind_test$test_sphere_acc))
    ind_test_sphere_logloss_lst <- c(ind_test_sphere_logloss_lst, mean(result$ind_test$test_sphere_log))

    if (length(result$ind_test$test_sphere_acc) != 1){
        ind_test_sphere_acc_var <- c(ind_test_sphere_acc_var, var(result$ind_test$test_sphere_acc)* length(test_indx)/total_length)
        ind_test_sphere_logloss_var <- c(ind_test_sphere_logloss_var, var(result$ind_test$test_sphere_log)* length(test_indx)/total_length)

    }
    
    ind_test_vesicle_acc_lst <- c(ind_test_vesicle_acc_lst, mean(result$ind_test$test_vesicle_acc))
    ind_test_vesicle_logloss_lst <- c(ind_test_vesicle_logloss_lst, mean(result$ind_test$test_vesicle_log))
    if (length(result$ind_test$test_vesicle_acc) != 1){
        ind_test_vesicle_acc_var <- c(ind_test_vesicle_acc_var, var(result$ind_test$test_vesicle_acc)*length(test_indx)/total_length)
        ind_test_vesicle_logloss_var <- c(ind_test_vesicle_logloss_var, var(result$ind_test$test_vesicle_log)*length(test_indx)/total_length)

    }
    
    ind_test_worm_acc_lst <- c(ind_test_worm_acc_lst, mean(result$ind_test$test_worm_acc))
    ind_test_worm_logloss_lst <- c(ind_test_worm_logloss_lst, mean(result$ind_test$test_worm_log))
    if (length(result$ind_test$test_worm_acc) != 1){
        ind_test_worm_acc_var <- c(ind_test_worm_acc_var, var(result$ind_test$test_worm_acc)*length(test_indx)/total_length)
        ind_test_worm_logloss_var <- c(ind_test_worm_logloss_var, var(result$ind_test$test_worm_log)*length(test_indx)/total_length)

    }
    
    ind_test_other_acc_lst <- c(ind_test_other_acc_lst, mean(result$ind_test$test_other_acc))
    ind_test_other_logloss_lst <- c(ind_test_other_logloss_lst, mean(result$ind_test$test_other_log))
    if (length(result$ind_test$test_other_acc) != 1){
        ind_test_other_acc_var <- c(ind_test_other_acc_var, var(result$ind_test$test_other_acc)*length(test_indx)/total_length)
        ind_test_other_logloss_var <- c(ind_test_other_logloss_var, var(result$ind_test$test_other_log)*length(test_indx)/total_length)

    }
}

Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 

Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 or 1 occurred”
Warning message in gam.fit3(x = args$X, y = args$y, sp = lsp, Eb = args$Eb, UrS = args$UrS, :
“fitted probabilities numerically 0 

```
data.frame(
    mean_train_accuracy = c(),
    std_train_accuracy = c(),
    
    mean_test_accuracy = c(),
    std_test_accuracy = c(),
    
    mean_train_logloss = c(),
    std_train_logloss = c(),
    
    mean_test_logloss = c(),
    std_test_logloss = c()
    )
```

In [10]:
num = nrow(data_index)
joint_train_acc_std = sd(joint_train_acc_var) 
ind_train_sphere_acc_std = sd(ind_train_sphere_acc_var) 
ind_train_vesicle_acc_std = sd(ind_train_vesicle_acc_var) 
ind_train_worm_acc_std = sd(ind_train_worm_acc_var) 
ind_train_other_acc_std = sd(ind_train_other_acc_var) 

joint_test_acc_std = sd(joint_test_acc_var) 
ind_test_sphere_acc_std = sd(ind_test_sphere_acc_var) 
ind_test_vesicle_acc_std = sd(ind_test_vesicle_acc_var) 
ind_test_worm_acc_std = sd(ind_test_worm_acc_var) 
ind_test_other_acc_std = sd(ind_test_other_acc_var) 

joint_train_logloss_std = sd(joint_train_logloss_var) 
ind_train_sphere_logloss_std = sd(ind_train_sphere_logloss_var) 
ind_train_vesicle_logloss_std = sd(ind_train_vesicle_logloss_var)
ind_train_worm_logloss_std = sd(ind_train_worm_logloss_var)
ind_train_other_logloss_std = sd(ind_train_other_logloss_var)

joint_test_logloss_std = sd(joint_test_logloss_var)
ind_test_sphere_logloss_std = sd(ind_test_sphere_logloss_var)
ind_test_vesicle_logloss_std = sd(ind_test_vesicle_logloss_var)
ind_test_worm_logloss_std = sd(ind_test_worm_logloss_var)
ind_test_other_logloss_std = sd(ind_test_other_logloss_var)


In [11]:
df.train <- data.frame(
    mean_train_accuracy = c(
                            mean(joint_train_acc_lst),
                            mean(ind_train_sphere_acc_lst),
                            mean(ind_train_vesicle_acc_lst),
                            mean(ind_train_worm_acc_lst),
                            mean(ind_train_other_acc_lst)
                            ),
    
    std_train_accuracy = c(
                            mean(joint_train_acc_std),
                            mean(ind_train_sphere_acc_std),
                            mean(ind_train_vesicle_acc_std),
                            mean(ind_train_worm_acc_std),
                            mean(ind_train_other_acc_std)
                           ),

    mean_test_accuracy = c(
                            mean(joint_test_acc_lst),
                            mean(ind_test_sphere_acc_lst),
                            mean(ind_test_vesicle_acc_lst),
                            mean(ind_test_worm_acc_lst),
                            mean(ind_test_other_acc_lst)
                           ),
    
    
    std_test_accuracy = c(
                            mean(joint_test_acc_std),
                            mean(ind_test_sphere_acc_std),
                            mean(ind_test_vesicle_acc_std),
                            mean(ind_test_worm_acc_std),
                            mean(ind_test_other_acc_std)
                           ),

    mean_train_logloss = c(
                            mean(joint_train_logloss_lst),
                            mean(ind_train_sphere_logloss_lst),
                            mean(ind_train_vesicle_logloss_lst),
                            mean(ind_train_worm_logloss_lst),
                            mean(ind_train_other_logloss_lst)
                           ),
    
    
    std_train_logloss = c(
                            mean(joint_train_logloss_std),
                            mean(ind_train_sphere_logloss_std),
                            mean(ind_train_vesicle_logloss_std),
                            mean(ind_train_worm_logloss_std),
                            mean(ind_train_other_logloss_std)
                           ),

    mean_test_logloss = c(
                            mean(joint_test_logloss_lst),
                            mean(ind_test_sphere_logloss_lst),
                            mean(ind_test_vesicle_logloss_lst),
                            mean(ind_test_worm_logloss_lst),
                            mean(ind_test_other_logloss_lst)
                           ),
    
    
    std_test_logloss = c(
                            mean(joint_test_logloss_std),
                            mean(ind_test_sphere_logloss_std),
                            mean(ind_test_vesicle_logloss_std),
                            mean(ind_test_worm_logloss_std),
                            mean(ind_test_other_logloss_std)
                           )
    )
rownames(df.train) <- c('joint', 'sphere', 'vesicle', 'worm', 'other')

In [12]:
df.train

,mean_train_accuracy,std_train_accuracy,mean_test_accuracy,std_test_accuracy,mean_train_logloss,std_train_logloss,mean_test_logloss,std_test_logloss
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
joint,0.6386067,0.002073293,0.6468170,0.001785036,1.05439007,0.030500308,3.1146556,2.228330309
sphere,0.8583640,0.002833019,0.8249150,0.001583052,0.35193855,0.011727217,0.5244771,0.096370447
vesicle,0.8820975,0.004238259,0.8867347,0.002057975,0.26249718,0.019133039,0.9663724,0.414440613
worm,0.8160689,0.001879460,0.8571699,0.001868030,0.40382459,0.002732153,0.4027283,0.008308042
other,0.9867013,0.001305579,0.9644983,0.001235826,0.03612976,0.001374028,1.2210777,1.515344174


In [13]:
write.csv(df.train, 'extra_GAM_result.csv', row.names = TRUE)